In [1]:
import os
import numpy as np
from skimage import exposure

### Normalization Function

In [ ]:
def normalize_image(image):
    """Normalize the image values to the range [0, 1]."""
    return (image - np.min(image)) / (np.max(image) - np.min(image))

In [ ]:
def percentile_based_histogram_equalization(image, mask=None):
    """
    Apply histogram equalization to the region of interest defined by a mask,
    excluding pixels with a value of 0 and using the 2% and 98% percentiles.
    
    Args:
        image (numpy.ndarray): Input image with values normalized between [0, 1].
        mask (numpy.ndarray, optional): Boolean mask of the same shape as the image, 
                                         where True indicates the region of interest. 
                                         If None, the entire image is used.
    
    Returns:
        numpy.ndarray: Image with histogram equalization applied based on percentiles.
    """
    # If no mask is provided, apply to the entire image
    if mask is None:
        mask = np.ones_like(image, dtype=bool)
    
    # Exclude pixels with a value of 0
    valid_mask = (image != 0) & mask
    print("valid_mask",valid_mask.shape)

    
    # Extract the non-zero values within the mask
    valid_values = image[valid_mask]
    print("valid",valid_values.shape)
    
    if valid_values.size == 0:
        raise ValueError("No valid pixels found for histogram equalization.")

    # Calculate the 5th, and 95th percentiles
    p2 = np.percentile(valid_values, 2)
    print("p2",p2)

    p98 = np.percentile(valid_values, 98)    
    print("p98",p98)


    # Rescale the values within the range [p2, p98] to [0, 1]
    rescaled_image = np.zeros_like(image)
    rescaled_image[valid_mask] = np.clip((valid_values - p2) / (p98 - p2), 0, 1)
    
    # Apply histogram equalization to the rescaled region
    equalized_rescaled = exposure.equalize_hist(rescaled_image[valid_mask])
    
    # Assign the equalized values back to the valid region
    equalized_image = image.copy()
    equalized_image[valid_mask] = equalized_rescaled

    # Retain original zero values
    equalized_image[~valid_mask] = 0
    
    return equalized_image

In [ ]:
def process_slices(input_dir, output_dir):
    """
    Process all slices in the input directory, applying percentile-based histogram equalization
    based on file name prefixes and save the modified slices to the output directory.
    """
    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Loop over each .npy file in the input directory
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.npy'):
            input_path = os.path.join(input_dir, file_name)
            
            # Load the slice
            slice_data = np.load(input_path)
            
            # Determine if the file has a contour or not
            if slice_data.ndim == 3 and slice_data.shape[2] == 2:
                # Process the image channel ([:, :, 0])
                image = slice_data[:, :, 0]
            elif slice_data.ndim == 3 and slice_data.shape[2] == 1:
                # Process the single-channel image
                image = slice_data[:, :, 0]
            else:
                print(f"Skipping {input_path}: Unexpected dimensions {slice_data.shape}")
                continue
            
           # Normalize and equalize the image
            normalized_image = normalize_image(image)
            print((np.unique(normalized_image)).shape)

            equalized_image = percentile_based_histogram_equalization(normalized_image)
            print((np.unique(equalized_image)).shape)


            # Update the slice with the modified image channel
            slice_data[:, :, 0] = equalized_image
            
            # Save the modified slice to the output directory
            output_path = os.path.join(output_dir, file_name)
            np.save(output_path, slice_data)
            print(f"Processed and saved: {output_path}")
            

### DEFINE YOUR INPUT/OUTPUT DIRECTORY

In [ ]:
input_directory = r""  # Replace with the path to your input directory
output_directory = r""  # Replace with the path to your output directory
process_slices(input_directory, output_directory)